# **Scraping Berita Detik.com**

## Import Library

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Fungsi Mengambil data yang ada web detik.com

In [ ]:
def get_data(url, category):
  data_page = requests.get(url)
  print(url)
  if data_page.status_code == 200:
    data_berita = []
    container = BeautifulSoup(data_page.content, 'html.parser')
    container_page = container.find('div', class_='detail__body-text itp_bodycontent')
    data = container_page.find_all('p')
    judul_berita = container.find('h1', class_='detail__title').text.strip()
    tanggal_berita = container.find('div', class_='detail__date').text.strip()

    data_content = ''.strip()
    for i in data:
      if  'ADVERTISEMENT' not in i.text.strip() and  'SCROLL TO RESUME CONTENT' not in i.text.strip() and i.find('div', class_='detail__multiple') == None:
        data_content += i.text.strip() + ' '

    data_berita.append(judul_berita)
    data_berita.append(tanggal_berita)
    data_berita.append(data_content)
    data_berita.append(category)

    return data_berita

## Fungsi Mengambil content yang ada di web detik.com

In [ ]:
def get_content (url, category):
  menu_page = requests.get(url)
  if menu_page.status_code == 200:
    content = BeautifulSoup(menu_page.content, 'html.parser')
    content_page = content.find('div', class_='grid-row list-content')
    container_content = content_page.find_all('div', class_='media media--left media--image-radius block-link')

    # Untuk menyimpan url content
    data_content = []
    for c in container_content:
      link_content = c.find('a', class_='media__link')
      subtitle = c.find('h2', class_='media__subtitle')
      if subtitle == None and '20.detik' not in str(link_content) and 'intermeso' not in str(link_content) and 'foto' not in str(link_content):
        data_berita = get_data(link_content.get('href'), category)
        data_content.append(data_berita)

    next_page = content.find_all('a', class_='pagination__item')
    for i in next_page:
      if 'Next' == i.text.strip():
        return (data_content, i.get('href'))

## Fungsi Mengambil Judul dan URL berita yang ada di detik.com


In [ ]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 1, category= ['Edu']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Detik.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

## Memanggil Fungsi `scraping_detik_news()`


Dokumentasi
```
scraping_detik_news(url='https://news.detik.com/indeks', num_content=1, category=['Edu'])
```
Dalam penggunaan fungsi scraping ini terdapat 3 parameter:
- Url : Link Website news.detik.com/index
- num_content : Jumlah Page pada website
- category : Category apa saja yang ingin di ambil


In [ ]:
scraping_detik_news()